In [1]:
import numpy as np
import pandas as pd
from glob import glob
import datetime
from pyproj import Transformer
import matplotlib.pyplot as plt
import wradlib as wrl
import re
import geopandas as gpd
import pygrib
from tqdm import tqdm

In [2]:
DIR_Harmony = 'F:/harmonie'
DIR_Harmony_save = 'F:/harmony processed/'

In [3]:
glob('F:/harmonie/*')

['F:/harmonie\\201510',
 'F:/harmonie\\201511',
 'F:/harmonie\\201512',
 'F:/harmonie\\201601',
 'F:/harmonie\\201602',
 'F:/harmonie\\201603',
 'F:/harmonie\\201604',
 'F:/harmonie\\201605',
 'F:/harmonie\\201606',
 'F:/harmonie\\201607',
 'F:/harmonie\\201608',
 'F:/harmonie\\201609',
 'F:/harmonie\\201610',
 'F:/harmonie\\201611',
 'F:/harmonie\\201612',
 'F:/harmonie\\201701',
 'F:/harmonie\\201702',
 'F:/harmonie\\201703',
 'F:/harmonie\\201704',
 'F:/harmonie\\201705',
 'F:/harmonie\\201706',
 'F:/harmonie\\201707',
 'F:/harmonie\\201708',
 'F:/harmonie\\201709',
 'F:/harmonie\\201710',
 'F:/harmonie\\201711',
 'F:/harmonie\\201712',
 'F:/harmonie\\201801',
 'F:/harmonie\\201802',
 'F:/harmonie\\201803',
 'F:/harmonie\\201804',
 'F:/harmonie\\201805',
 'F:/harmonie\\201806',
 'F:/harmonie\\201807',
 'F:/harmonie\\201808',
 'F:/harmonie\\201809',
 'F:/harmonie\\201810',
 'F:/harmonie\\201811',
 'F:/harmonie\\201812',
 'F:/harmonie\\201901',
 'F:/harmonie\\201902',
 'F:/harmonie\\2

In [4]:
urls = np.array(glob(DIR_Harmony + '/*/*'))

df_meta = pd.DataFrame(urls,columns=['url'])

timestring_start = len('2015100100.00')
timestring_end = len('.00')
extract_timestring = lambda x : x[-timestring_start:-timestring_end]
extract_timestamp = lambda x : datetime.datetime(int(x[:4]),int(x[4:6]),int(x[6:8]),int(x[8:10]))
timestamps = np.array([extract_timestamp(extract_timestring(u)) for u in urls])
df_meta.loc[:,'datetime'] = timestamps
df_meta.loc[:,'pred dist'] = df_meta.loc[:,'url'].str[-2:].astype(int)

In [5]:
tmp_a = np.array([extract_timestring(u) for u in urls])
# tmp_b = np.array([extract_timestamp(u) for u in tmp_a])

In [6]:
def read_parameter_info(parameter_list, param_number):
    ''' Takes a number for the parameter of interest, 
        returns various information incl. gridded values '''
    parameter_list[param_number].dataDate
    parameter_list[param_number].parameterName
    parameter_list[param_number].indicatorOfTypeOfLevel
    parameter_list[param_number].typeOfLevel
    parameter_list[param_number].latitudes
    parameter_list[param_number].longitudes
    parameter_list[param_number].values
    
    parameter_list[param_number].Ni
    parameter_list[param_number].Nj
    
    
    lats = parameter_list[param_number].latitudes.reshape(parameter_list[param_number].Nj, parameter_list[param_number].Ni)
    lons = parameter_list[param_number].longitudes.reshape(parameter_list[param_number].Nj, parameter_list[param_number].Ni)
    grid_values = parameter_list[param_number].values.reshape(parameter_list[param_number].Nj, parameter_list[param_number].Ni)
    
    
    param_output = {"date": parameter_list[param_number].dataDate,
                    "grib_number": parameter_list[param_number].parameterName,
                    "indicator_type": parameter_list[param_number].indicatorOfTypeOfLevel,
                    "type_of_level": parameter_list[param_number].indicatorOfTypeOfLevel,
                    "lats": lats,
                    "lons": lons,
                    "values": grid_values}
    return(param_output)
def nwp_plot(rain_array, lons, lats, bg_map_file, plot_title):
    ''' Function that plots gridded NWP data'''
    bg_map = gpd.read_file(bg_map_file)
    
    # create custom color map
    #cmap = colors.ListedColormap(["#85E3E4", '#42D8D8', '#42AFD8', '#4282D8', "#FFE600", '#FFAF00', '#FF5050', '#FF1A1A', "#BD0000", "#8C0000"])
    #boundaries = [0, .5, 1, 2, 3, 4, 5, 7.5, 10, 15, 20]
    #norm = colors.BoundaryNorm(boundaries, cmap.N, clip=True)
    
    bg_map.plot(facecolor="lightgrey")
    plt.pcolor(lons, lats, rain_array, shading = "auto", alpha=0.8)#, cmap=cmap, norm=norm)
    cbar = plt.colorbar(fraction=0.046, pad=0.04)
    #cbar.ax.set_ylabel('Rainfall intensity [mm/h]', rotation=90)
    #plt.xlim(7,15) # longitudes for Denmark (for a zoomed plot)
    #plt.ylim(54.5,58) # lattitudes for Denmark (for a zoomed plot)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title(plot_title)

In [7]:
df_meta = df_meta.sort_values(['datetime','pred dist'])


In [8]:
# df_meta

In [9]:
# Monthly indicators for filtering
datetime_series = pd.DatetimeIndex(df_meta.datetime)
months = ['%02d' % i for i in datetime_series.month]
years = [str(i) for i in datetime_series.year]
df_meta.loc[:,'year and month'] = [a+b for a,b in zip(years,months)]
df_start_end = pd.concat([df_meta.groupby('year and month').min().loc[:,'datetime'],df_meta.groupby('year and month').max().loc[:,'datetime']],axis=1)
df_start_end.columns = ['start','end']
df_meta_group_lists = df_meta.groupby('datetime').apply(lambda x: list(x.index))


In [10]:
import pygrib
tmp = df_meta.iloc[0,0]


In [11]:
# initialization so the variables are defined outside the scope of the for loop

raw_data_list = []
# one month at a time for easier debugging
for index,row in df_start_end.loc['201511':].iterrows():
    # print(index)
    URL_data = f'{DIR_Harmony_save}{index}_data.npy'
    URL_urls = f'{DIR_Harmony_save}{index}_urls.npy'
    URL_shapes = f'{DIR_Harmony_save}{index}_shapes.npy'
    print(URL_data)
    
    if len(glob(URL_data)) > 0:
        continue
    
    # timestamps for first and last images of month
    start = row['start']
    end = row['end']
    
    # Subset of month
    monthly_subset = df_meta_group_lists.loc[start:end]
    len_subset = len(monthly_subset)
    # Iterate over radar slices
    
    pbar = tqdm(range(len_subset))
    pbar.set_description("Processing %s" % index)
    monthly_data = []
    urls = []
    shapes = []
    for indx,single_scan_indexes in zip(pbar,monthly_subset):
        single_scan = df_meta.loc[single_scan_indexes]
        tmp_urls = single_scan.loc[:,'url'].values
        
        # Can crash if a file is corrupted
        try:
            data = np.array([pygrib.open(i).read()[0].values for i in tmp_urls])
        except KeyboardInterrupt: # for early stopping
            assert False
        except: # For other kinds of errors
            problematic_indexes.append(single_scan_indexes)
            continue # just skip the whole thing
        
        shapes.append(data.shape)
        
        if data.shape != (67, 469, 489):
            data_tmp = np.zeros((67, 469, 489))
            f,xr,yr = data.shape
            data_tmp[:f,:xr,:yr] = data
            data = data_tmp
            
        
        monthly_data.append(data)
        urls.append(tmp_urls)
    monthly_data_tensor = np.array(monthly_data,dtype='float32')
    
    np.save(URL_data,monthly_data_tensor)
    np.save(URL_urls,urls)
    np.save(URL_shapes,np.array(shapes))

F:/harmony processed/201511_data.npy
F:/harmony processed/201512_data.npy
F:/harmony processed/201601_data.npy
F:/harmony processed/201602_data.npy
F:/harmony processed/201603_data.npy
F:/harmony processed/201604_data.npy
F:/harmony processed/201605_data.npy
F:/harmony processed/201606_data.npy
F:/harmony processed/201607_data.npy
F:/harmony processed/201608_data.npy
F:/harmony processed/201609_data.npy
F:/harmony processed/201610_data.npy
F:/harmony processed/201611_data.npy
F:/harmony processed/201612_data.npy
F:/harmony processed/201701_data.npy
F:/harmony processed/201702_data.npy
F:/harmony processed/201703_data.npy
F:/harmony processed/201704_data.npy
F:/harmony processed/201705_data.npy
F:/harmony processed/201706_data.npy
F:/harmony processed/201707_data.npy
F:/harmony processed/201708_data.npy
F:/harmony processed/201709_data.npy
F:/harmony processed/201710_data.npy
F:/harmony processed/201711_data.npy
F:/harmony processed/201712_data.npy
F:/harmony processed/201801_data.npy
F

Processing 202006: 100%|██████████| 118/118 [01:04<00:00,  1.83it/s]
C:\Users\sverrirhd\Anaconda3\envs\gis_wradlib_torch\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


F:/harmony processed/202007_data.npy


Processing 202007: 100%|██████████| 124/124 [01:51<00:00,  1.11it/s]


F:/harmony processed/202008_data.npy


...

Processing 202010: 100%|██████████| 124/124 [02:31<00:00,  1.22s/it]


F:/harmony processed/202011_data.npy


Processing 202011: 100%|██████████| 120/120 [02:01<00:00,  1.01s/it]


F:/harmony processed/202012_data.npy


Processing 202012: 100%|██████████| 119/119 [02:18<00:00,  1.16s/it]
